In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import pandas as pd
import requests
import random
import time
import os

current_dir = os.getcwd()

firefox_exe = f"{current_dir}/geckodriver"
chrome_exe = f"{current_dir}/chromedriver"

In [2]:
# Initialize WebDriver
service = Service(executable_path=firefox_exe)

options = Options()
options.set_preference("general.smoothScroll", True)

driver = webdriver.Firefox(service=service, options=options)
driver.get("https://defillama.com/")

# Wait for page content to load
driver.maximize_window()
time.sleep(2)

actions = ActionChains(driver)

for _ in range(6):
    actions.scroll_by_amount(0, 300).perform()
    time.sleep(.5)

In [3]:
links = set()

while len(links) < 3590:
    divs = driver.find_elements(By.CLASS_NAME, "row")

    for div in divs:
        link = div.find_element(By.TAG_NAME, "a").get_attribute("href")
        links.add(link)
        
    actions.scroll_by_amount(0, 500).perform()
    time.sleep(1)

driver.quit()

num_links = len(links)

print(f"Total links found: {num_links}")

Total links found: 3591


In [4]:
with open ("protocol_links.txt", "w") as f:
    for link in links:
        f.write(f"{link}\n")

del links

In [5]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

In [6]:
links = []

with open("protocol_links.txt", "r") as f:
    for i in range(num_links):
        links.append(f.readline().strip())

In [7]:
response = requests.get(
    url = 'https://headers.scrapeops.io/v1/user-agents',
    params = {
        'api_key': '',
        'num_results': '96'
    }
)

results = response.json().get("result", ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36 Edg/90.0.818.46"])

In [8]:
recheck_links = []
df = pd.DataFrame({}, columns=["url", "protocol", "twitter_handle"])

In [9]:
crawled = df.shape[0]

for i, link in enumerate(links[crawled:], start=crawled):

    service = Service(executable_path=chrome_exe)
    options = Options()

    options.add_argument('--headless=new')
    options.add_argument(f"user-agent={results[random.randint(0, len(results) -1)]}")
    options.add_argument('--blink-settings=imagesEnabled=false')

    driver = webdriver.Chrome(service=service, options=options)
    driver.get(link)

    driver.maximize_window()

    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".sc-289dd4cb-6.eksiYR"))
        )

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".sc-528311f8-1.hBsCiU"))
        )

        anchor_elements = driver.find_element(By.CSS_SELECTOR, ".sc-289dd4cb-6.eksiYR").find_elements(By.TAG_NAME, 'a')

        for a in anchor_elements:
            if a.find_element(By.TAG_NAME, "span").text == "Twitter":
                twitter_link = a.get_attribute("href")
                break
        
        protocol = driver.find_element(By.CSS_SELECTOR, ".sc-528311f8-1.hBsCiU").find_element(By.TAG_NAME, "span").text

        df.loc[i] = [driver.current_url, protocol, twitter_link]

    except Exception as e:
        recheck_links.append(link)

        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".sc-528311f8-1.hBsCiU"))
            )

            protocol = driver.find_element(By.CSS_SELECTOR, ".sc-528311f8-1.hBsCiU").find_element(By.TAG_NAME, "span").text
        except Exception as e:
            protocol = "missing"

        df.loc[i] = [driver.current_url, protocol, "missing"]
    finally:
        driver.quit()

In [10]:
df.to_csv("defillama_twitter_handles.csv")